In [1]:
import sklearn
import time
import pandas as pd   
from matplotlib import pyplot
from numpy import mean
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB, CategoricalNB
import numpy as np



In [11]:
def robust_scale(df):
    numerical_columns = df.select_dtypes(include=['number']).columns
    scaler = RobustScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

    return df

def knn_impute_numerical_columns(df, n_neighbors=100):
    df_imputed = df.copy()
    df_imputed = pd.get_dummies(drop_first=False, data=df_imputed)
    
    categorical_columns = df_imputed.select_dtypes(exclude=['number']).columns
    numerical_cols = df_imputed.select_dtypes(include=['number']).columns
    
    imputer = KNNImputer(n_neighbors=n_neighbors)
    # imputer = SimpleImputer(strategy='mean') 
    df_imputed[numerical_cols] = imputer.fit_transform(df_imputed[numerical_cols])
    
    # Label encode categorical columns
    # label_encoders = {}
    # for col in categorical_columns:
    #     label_encoders[col] = LabelEncoder()
    #     df_imputed[col] = label_encoders[col].fit_transform(df_imputed[col])

    # imputer = SimpleImputer(strategy='constant', fill_value='Unknown')

    
    # df_imputed[categorical_columns] = imputer.fit_transform(df_imputed[categorical_columns])
    # cat_imputer = SimpleImputer(strategy='most_frequent')
    # df_imputed[categorical_columns] = cat_imputer.fit_transform(df_imputed[categorical_columns])

    return df_imputed


In [10]:
df = pd.read_csv('train.csv')
# df

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

df = pd.read_csv('train.csv')

# Load your dataset into a DataFrame (assuming it's named 'df')
# Perform data preprocessing here

# Split the dataset into features (X) and the target variable (y)
X = df.loc[:, df.columns != "hospital_death"]
y = df[["hospital_death"]]


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess categorical columns (one-hot encoding)
categorical_cols = df.select_dtypes(exclude=['number']).columns
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_train_cat_encoded = encoder.fit_transform(X_train[categorical_cols])
X_test_cat_encoded = encoder.transform(X_test[categorical_cols])

# Preprocess numerical columns (standard scaling)
numerical_cols = df.select_dtypes(include=['number']).columns
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train[numerical_cols])
X_test_num_scaled = scaler.transform(X_test[numerical_cols])

# Combine encoded categorical and scaled numerical features
X_train_processed = pd.concat([pd.DataFrame(X_train_cat_encoded), pd.DataFrame(X_train_num_scaled)], axis=1)
X_test_processed = pd.concat([pd.DataFrame(X_test_cat_encoded), pd.DataFrame(X_test_num_scaled)], axis=1)

# Build and train an MLPClassifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', max_iter=100, random_state=42)
mlp_classifier.fit(X_train_processed, y_train)

# Predict on the testing data
y_pred = mlp_classifier.predict(X_test_processed)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, mlp_classifier.predict_proba(X_test_processed)[:, 1])

print("Accuracy:", accuracy)
print("ROC AUC:", roc_auc)


/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


KeyError: "['hospital_death'] not in index"